In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [4]:
data = requests.get(standings_url)

In [16]:
#retirando apenas os links dos times da tabela
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [18]:
#tornando os links em formato completo
team_urls = [f"https://fbref.com{l}" for l in links]

In [22]:
#pegando informação dos times
team_url = team_urls[0]
data = requests.get(team_url)
matches = pd.read_html(data.text, match="Scores & Fixtures")
matches

C:\Users\Carlos\AppData\Local\Temp\ipykernel_11836\195754956.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")


AttributeError: 'list' object has no attribute 'shape'